In [1]:
from tqdm import tqdm
import csv
import numpy as np
from src.config import TLCPaths

UMLS_KEY = "43f9234c-4977-45f6-a440-2dda1b43d919"

In [6]:
### load wumls 
wumls_file = '/home/tim/MedicalLay/WUMLS/MRCONSO_WUMLS_GER.RRF'

wumls_names = {}
with open(wumls_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='|')
    for row in spamreader:
        cui = row[0]
        name = row[-5]
        wumls_names[name] = cui
print(len(wumls_names))


184475


In [24]:
with open(TLCPaths.project_data_path.joinpath("wumls_names_and_cuis.csv"),"w", newline="") as f:
    spamwriter = csv.writer(f, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for name, cui in wumls_names.items():
        spamwriter.writerow([cui, name])

In [11]:
german_umls_names = {}
with open(wumls_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='|')
    for row in spamreader:
        if  row[-8] != "WIKTIONARY":
            cui = row[0]
            name = row[-5]
            german_umls_names[name] = cui
            
print(len(german_umls_names))

with open(TLCPaths.project_data_path.joinpath("german_umls_names_and_cuis.csv"),"w", newline="") as f:
    spamwriter = csv.writer(f, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for name, cui in german_umls_names.items():
        spamwriter.writerow([cui, name])

182008


In [5]:
# get wumls and english names
# encode all english names and build index
# encode query and find nearest neighbour from index

In [15]:
### load wumls and english names
wumls_file = '/home/tim/MedicalLay/WUMLS/MRCONSO_WUMLS_GER.RRF'

cuis = set()
with open(wumls_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='|')
    for row in spamreader:
        cui = row[0]
        cuis.add(cui)
        if cui == "C0242362":
            print(row)

print(len(cuis))

['C0242362', 'GER', 'P', 'L1247100', 'PF', 'S1489046', 'Y', 'A13455396', '', 'M0011561', 'D007405', 'MSHGER', 'PEP', 'D007405', 'Diskushernie', '3', 'N', '', '']
['C0242362', 'GER', 'S', 'L1236418', 'PF', 'S1478364', 'N', 'A25863238', '', '', '10050296', 'MDRGER', 'LLT', '10022638', 'Bandscheibenvorfall', '3', 'N', '', '']
['C0242362', 'GER', 'S', 'L1236418', 'PF', 'S1478364', 'Y', 'A7484171', '', 'M0011561', 'D007405', 'MSHGER', 'EP', 'D007405', 'Bandscheibenvorfall', '3', 'N', '', '']
['C0242362', 'GER', 'S', 'L1289264', 'PF', 'S1531210', 'Y', 'A7504682', '', 'M0011561', 'D007405', 'MSHGER', 'EP', 'D007405', 'Prolabierte Bandscheibe', '3', 'N', '', '']
['C0242362', 'GER', 'S', 'L13322943', 'PF', 'S16286832', 'Y', 'A26916538', '', '', '10050296', 'MDRGER', 'LLT', '10078023', 'Extrusion einer Bandscheibe', '3', 'N', '', '']
['C0242362', 'GER', 'S', 'L5031305', 'PF', 'S5718810', 'N', 'A25915339', '', '', '10050296', 'MDRGER', 'LLT', '10050296', 'Bandscheibenprotrusion', '3', 'N', '', ''

In [17]:
import sqlite3

db_path = "/home/tim/PycharmProjects/py-umls/databases/umls.db"
db = sqlite3.connect(db_path)
cur = db.cursor()

In [18]:
for res in cur.execute("SELECT * FROM descriptions WHERE LAT = 'ENG' LIMIT 5"):
    print(res)

('C0000005', 'ENG', 'MSH', 'PEP', '(131)I-Macroaggregated Albumin', 'T116|T121|T130')
('C0000039', 'ENG', 'MTH', 'PN', '1,2-dipalmitoylphosphatidylcholine', 'T109|T121')
('C0000039', 'ENG', 'MSH', 'MH', '1,2-Dipalmitoylphosphatidylcholine', 'T109|T121')
('C0000039', 'ENG', 'MSH', 'PM', '1,2 Dipalmitoylphosphatidylcholine', 'T109|T121')
('C0000052', 'ENG', 'MTH', 'PN', '1,4-alpha-Glucan Branching Enzyme', 'T116|T126')


In [19]:
cui = "C0005903"
def get_prefered_name_of_cui(cuis):
    cui_names_mapping = {}
    not_found = 0
    for identifier in tqdm(cuis):
        sql = f"SELECT STR FROM descriptions WHERE CUI = ? ORDER BY CASE WHEN SAB = 'MTH' THEN 1 WHEN SAB = 'SNOMEDCT' THEN 2  WHEN SAB = 'MSH' THEN 3 ELSE 4 END LIMIT 1"
        
        try:
            res = next(cur.execute(sql, (identifier,)))[0]
            cui_names_mapping[identifier] = res
        except StopIteration:
            # print('Search term ' + "'" + str(identifier) + "'" + ' not found')
            not_found += 1
    print(f"{not_found} not found from {len(cuis)}")
    return cui_names_mapping

cui_names_mapping = get_prefered_name_of_cui(cuis)

100%|██████████| 110121/110121 [00:02<00:00, 42505.20it/s]

574 not found from 110121


In [20]:
get_prefered_name_of_cui(["C0242362"])

100%|██████████| 1/1 [00:00<00:00, 1046.22it/s]

1 not found from 1


{}

In [10]:
concept_cuis, concept_names = zip(*cui_names_mapping.items())

In [12]:
with open(TLCPaths.project_data_path.joinpath("english_wumls.csv"),"w", newline="") as f:
    spamwriter = csv.writer(f, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for cui, name in cui_names_mapping.items():
        spamwriter.writerow([cui, name])